# Языковое моделирование

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install pymorphy2
!pip install stop-words

     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 5.1 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=97adcb1b03cbe37c66ff7b8c5d12f1e0332c57f76c60562fba8f4366362712b6
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [50]:
import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
import tensorflow as tf
import os

In [51]:
path_to_file = '/content/drive/MyDrive/evgenyi_onegin.txt'

In [52]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [53]:
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [54]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [55]:
text = text.split('\n\n')

In [56]:
text[14]

'                        Мы все учились понемногу\n                        Чему-нибудь и как-нибудь,\n                        Так воспитаньем, слава богу,\n                        У нас немудрено блеснуть.\n                        Онегин был по мненью многих\n                        (Судей решительных и строгих)\n                        Ученый малый, но педант:\n                        Имел он счастливый талант\n                        Без принужденья в разговоре\n                        Коснуться до всего слегка,\n                        С ученым видом знатока\n                        Хранить молчанье в важном споре\n                        И возбуждать улыбку дам\n                        Огнем нежданных эпиграмм.'

In [57]:
len(text)

782

In [58]:
text_only = []
for strofa in text:
    if len(strofa) < 350:
        continue
    else:
        text_only.append(strofa)
len(text_only)

376

In [59]:
data = pd.DataFrame(text_only)
data = data.rename(columns={0: "text"})
data

,text
0,Не мысля гордый свет з...
1,"""Мой дядя самых честны..."
2,Так думал молодой пове...
3,Служив отлично благоро...
4,Когда же юности мятежн...
...,...
371,А счастье было так воз...
372,Она ушла. Стоит Евгени...
373,"Кто б ни был ты, о мой..."
374,"Прости ж и ты, мой спу..."


In [60]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [61]:
def exclude_punctuation(txt):
    txt = "".join(c for c in txt if c not in exclude)    
    txt = re.sub("\n", " \n", txt)    
    return txt

In [62]:
def preprocess_text(txt, morph = False):
    txt = str(txt)
    txt = txt.lower()
    txt = re.sub("\n", "zzz", txt)
    new_txt =[]
    for word in txt.split():
        if word == "zzz":
            word = " \n"
            
        else:
            if morph:
                word = morpher.parse(word)[0].normal_form
            else:
                pass
        new_txt.append(word)

    return new_txt

In [63]:
data['text_splited'] = data['text'].apply(exclude_punctuation)
data['text_splited'] = data['text_splited'].apply(preprocess_text, morph = False)

data

,text,text_splited
0,Не мысля гордый свет з...,"[не, мысля, гордый, свет, забавить, \n, внима..."
1,"""Мой дядя самых честны...","[мой, дядя, самых, честных, правил, \n, когда..."
2,Так думал молодой пове...,"[так, думал, молодой, повеса, \n, летя, в, пы..."
3,Служив отлично благоро...,"[служив, отлично, благородно, \n, долгами, жи..."
4,Когда же юности мятежн...,"[когда, же, юности, мятежной, \n, пришла, евг..."
...,...,...
371,А счастье было так воз...,"[а, счастье, было, так, возможно, \n, так, бл..."
372,Она ушла. Стоит Евгени...,"[она, ушла, стоит, евгений, \n, как, будто, г..."
373,"Кто б ни был ты, о мой...","[кто, б, ни, был, ты, о, мой, читатель, \n, д..."
374,"Прости ж и ты, мой спу...","[прости, ж, и, ты, мой, спутник, странный, \n..."


In [64]:
def get_w2i_i2w(column_data):
    
    dump = list(column_data.values)
    dump_txt_split = []
    for sublist in dump:
        for item in sublist:
            dump_txt_split.append(item)


    vocab = sorted(set(dump_txt_split))           
            
    # Creating a mapping from unique characters to indices
    word2idx = {u:i for i, u in enumerate(vocab)}
    idx2word = np.array(vocab)
    
    print(len(vocab), len(word2idx), len(idx2word))
    return word2idx, idx2word

In [65]:
word2idx, idx2word = get_w2i_i2w(data['text_splited'])        
data['int_text_splited'] = data['text_splited'].apply(lambda x: [word2idx[c] for c in x])

8427 8427 8427


In [66]:
data

,text,text_splited,int_text_splited
0,Не мысля гордый свет з...,"[не, мысля, гордый, свет, забавить, \n, внима...","[3817, 3634, 1358, 6327, 2071, 0, 878, 1844, 9..."
1,"""Мой дядя самых честны...","[мой, дядя, самых, честных, правил, \n, когда...","[3487, 1912, 6292, 8129, 5394, 0, 2788, 3817, ..."
2,Так думал молодой пове...,"[так, думал, молодой, повеса, \n, летя, в, пы...","[7249, 1867, 3505, 4930, 0, 3102, 565, 5872, 3..."
3,Служив отлично благоро...,"[служив, отлично, благородно, \n, долгами, жи...","[6660, 4535, 325, 0, 1720, 2041, 1919, 4512, 0..."
4,Когда же юности мятежн...,"[когда, же, юности, мятежной, \n, пришла, евг...","[2788, 1980, 8363, 3638, 0, 5651, 1915, 5222, ..."
...,...,...,...
371,А счастье было так воз...,"[а, счастье, было, так, возможно, \n, так, бл...","[98, 7202, 550, 7249, 924, 0, 7249, 380, 4072,..."
372,Она ушла. Стоит Евгени...,"[она, ушла, стоит, евгений, \n, как, будто, г...","[4371, 7870, 7006, 1914, 0, 2637, 515, 1462, 5..."
373,"Кто б ни был ты, о мой...","[кто, б, ни, был, ты, о, мой, читатель, \n, д...","[2960, 160, 4037, 546, 7548, 4141, 3487, 8153,..."
374,"Прости ж и ты, мой спу...","[прости, ж, и, ты, мой, спутник, странный, \n...","[5751, 1951, 2446, 7548, 3487, 6916, 7049, 0, ..."


In [67]:
def get_all_int(column_data):
    
    int_dump = column_data.values
    all_txt_as_int = []

    for sublist in int_dump:
        for item in sublist:
            all_txt_as_int.append(item)
    all_txt_as_int = np.array(all_txt_as_int)    
    
    return all_txt_as_int

all_txt_as_int = get_all_int(data['int_text_splited'])

In [68]:
# The maximum length sentence you want for a single input in characters
seq_length = 50
examples_per_epoch = len(all_txt_as_int)//(seq_length+1)

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(all_txt_as_int)

for i in word_dataset.take(15):
    print(idx2word[i.numpy()])

не
мысля
гордый
свет
забавить
 

вниманье
дружбы
возлюбя
 

хотел
бы
я
тебе
представить


In [69]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

In [70]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [71]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'не мысля гордый свет забавить  \n вниманье дружбы возлюбя  \n хотел бы я тебе представить  \n залог достойнее тебя  \n достойнее души прекрасной  \n святой исполненной мечты  \n поэзии живой и ясной  \n высоких дум и простоты  \n но так и быть рукой пристрастной  \n прими собранье пестрых глав  \n'
Target data: 'мысля гордый свет забавить  \n вниманье дружбы возлюбя  \n хотел бы я тебе представить  \n залог достойнее тебя  \n достойнее души прекрасной  \n святой исполненной мечты  \n поэзии живой и ясной  \n высоких дум и простоты  \n но так и быть рукой пристрастной  \n прими собранье пестрых глав  \n полусмешных'


In [72]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 50), (64, 50)), types: (tf.int64, tf.int64)>

In [73]:
# Length of the vocabulary in chars
vocab_size = len(idx2word)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [74]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  
    inputs = tf.keras.layers.Input(batch_input_shape=[batch_size, None])

    x =     tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    print(x.shape)
    x1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.concatenate([x,x1], axis=-1)
    
    print(x.shape)
    x2 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.add([x,x2])
    
    print(x.shape)
    x3 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)   
    
    x = tf.keras.layers.add([x,x3])   
    x = tf.keras.layers.Dense(vocab_size)(x)
    print(x.shape)

    model =tf.keras. Model(inputs=inputs, outputs=x)
    
    return model

In [75]:
model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

(64, None, 128)
(64, None, 1152)
(64, None, 1152)
(64, None, 8427)


In [76]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 50, 8427) # (batch_size, sequence_length, vocab_size)


In [77]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(64, None)]         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (64, None, 128)      1078656     input_3[0][0]                    
__________________________________________________________________________________________________
gru_6 (GRU)                     (64, None, 1024)     3545088     embedding_2[0][0]                
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (64, None, 1152)     0           embedding_2[0][0]                
                                                                 gru_6[0][0]                

In [78]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 50, 8427)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.039446


In [79]:
model.compile(optimizer='adam', loss=loss)

In [80]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

In [81]:
EPOCHS = 200

In [82]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
8/8 [==============================] - 4s 174ms/step - loss: 8.7691
Epoch 2/200
8/8 [==============================] - 1s 172ms/step - loss: 7.7917
Epoch 3/200
8/8 [==============================] - 1s 172ms/step - loss: 7.0464
Epoch 4/200
8/8 [==============================] - 1s 172ms/step - loss: 6.9264
Epoch 5/200
8/8 [==============================] - 1s 172ms/step - loss: 6.8775
Epoch 6/200
8/8 [==============================] - 1s 174ms/step - loss: 6.8392
Epoch 7/200
8/8 [==============================] - 1s 175ms/step - loss: 6.7691
Epoch 8/200
8/8 [==============================] - 1s 173ms/step - loss: 6.6002
Epoch 9/200
8/8 [==============================] - 1s 175ms/step - loss: 6.4225
Epoch 10/200
8/8 [==============================] - 1s 177ms/step - loss: 6.2515
Epoch 11/200
8/8 [==============================] - 1s 175ms/step - loss: 6.0796
Epoch 12/200
8/8 [==============================] - 1s 175ms/step - loss: 5.8800
Epoch 13/200
8/8 [===================

In [83]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_200'

In [84]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

(1, None, 128)
(1, None, 1152)
(1, None, 1152)
(1, None, 8427)


In [85]:
def generate_text(model, start_string, tmprt):
    # Evaluation step (generating text using the learned model)
    
    start_string = exclude_punctuation(start_string)
    #print(start_string)
    start_string_asis = preprocess_text(start_string, morph = False)
    
    # Number of characters to generate
    num_generate = 30

    # Converting our start string to numbers (vectorizing)
    input_eval = [word2idx[s] for s in start_string_asis]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = tmprt
    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [86]:
print(generate_text(model, start_string=u"счастье ", tmprt=1.3))

счастье пожалуйте мне бедный милой  
 скрыв никто без двор  
 одно вид его на цельным окнам тени ходят  
 мелькают профили пустяков  
 с ним клубился смиренной  
 невинной прелести


In [87]:
start_string= 'А счастье было так '

In [88]:
print(generate_text(model, start_string=start_string, tmprt=1.3))

А счастье было так душеньки путь  
 нетерпеливому герою  
 отселе в думу погружен  
 глядел на грозный пламень он прощай свидетель падшей славы  
 петровский замок ну не стой  
 пошел уже столпы


In [89]:
print(generate_text(model, start_string=start_string, tmprt=1))

А счастье было так ступить  
 приподнялася грудь ланиты  
 мгновенным пламенем покрыты  
 дыханье замерло в устах  
 и в слухе шум и блеск в очах  
 настанет ночь луна обходит  
 дозором


In [92]:
print(generate_text(model, start_string=start_string, tmprt=0.001))

А счастье было так ступить  
 приподнялася грудь ланиты  
 мгновенным пламенем покрыты  
 дыханье замерло в устах  
 и в слухе шум и блеск в очах  
 настанет ночь луна обходит  
 дозором


In [91]:
print(generate_text(model, start_string=start_string, tmprt=5))

А счастье было так рооr пополам обретут альбом вслед ужели стремит татьяна бестолкова живит жизни моих лет полет  
 такою поясок дам хваленых небесных прелестный господа деле воскресла родном увы тани охлаждающая потопленные унижусь
